Ling 406 - Term Project - Marco Fonseca (Yelp reviews)

The code below imports the necessary library and then it does some data processing. Reviews that have 3 or more stars are classified as positive, and reviews that have 2 or 1 stars are classified as negative. Some data processing to delete unwanted characters ('\xa0') was also performed. The result was a list of lists. A for loop was desgined in order to create tuples in this lists of lists that group the review and its annotation together. Then, steps very similar to the movie reviews were conducted, with the difference that the "stars" in the beginning of the variable that is used to make the frequency distribution was made. 

In [1]:
import nltk
import random
import re

with open('all_reviews.txt', 'r') as file:
    data = file.read().replace('\n', '')

l= data.split(']]]')
l2 = [s.replace('[[[', '') for s in l]
l3 = [s.replace('{{{1star}}}', '{neg}') for s in l2]
l4 = [s.replace('{{{2star}}}', '{neg}') for s in l3]
l5 = [s.replace('{{{3star}}}', '{pos}') for s in l4]
l6 = [s.replace('{{{4star}}}', '{pos}') for s in l5]
l7 = [s.replace('{{{5star}}}', '{pos}') for s in l6]
l8 = [s.replace('\xa0', '') for s in l7]

out = []
for strings in l8: 
    s = re.match( r'(.*)\}(.*?) .*', strings, re.M|re.I)
    #print(s)
    if s : 
        first = s.group(1) + '}'
        second = strings.replace(first,'')
        tuple = ( second , first ) 
        out.append(tuple)
        
random.shuffle(out)
words = nltk.word_tokenize(data)

all_words = []

for w in words:
    all_words.append(w.lower())
    
all_words2 = nltk.FreqDist(all_words)
all_words3 = list(all_words2.keys())
stars =  {"{", "1star", "2star", "3star", "4star", "5star", "}", '[', ']'}
all_words4 = [e for e in all_words3 if e not in stars] #3rd set of features



The cells below show the features employed. They were the same as the movie reviews. 

In [2]:
baseline = all_words4
bigram_features = list(nltk.bigrams(baseline)) #2nd set of features
remove = {":", ",", ".", ";", "?", "!", "(", ")", "'", "``" "to", "a", "the", "an", "on", "at", "in"}
content_features = [e for e in baseline if e not in remove] #3rd set of features
bigrams_content_features =  list(nltk.bigrams(content_features))#4th set of features. 

The cells below have the functions that looks for the features, followed by the accuracies of each model. Steps were the same as the ones employed in the movie reviews, including 20% of the data for training data and 80% for testing data.

In [3]:
def find_features_baseline(document):
    words = set(document)
    features = {}
    for w in baseline:
        features[w] = (w in words)
    return features 

def find_features_bigrams(document):
    words = set(document)
    features = {}
    for w in bigram_features:
        features[w] = (w in words)    
    return features

def find_features_content(document):
    words = set(document)
    features = {}
    for w in content_features:
        features[w] = (w in words)    
    return features

def find_features_bigrams_content(document):
    words = set(document)
    features = {}
    for w in bigrams_content_features:
        features[w] = (w in words)    
    return features

featuresets_baseline = [(find_features_baseline(rev), category) for (rev, category) in out]
featuresets_bigrams = [(find_features_bigrams(rev), category) for (rev, category) in out]
featuresets_content = [(find_features_content(rev), category) for (rev, category) in out]
featuresets_bigrams_content = [(find_features_bigrams_content(rev), category) for (rev, category) in out]

In [4]:
training_set_baseline = featuresets_baseline[:2077]
testing_set_baseline = featuresets_baseline[:2077]
classifier_baseline = nltk.NaiveBayesClassifier.train(training_set_baseline)
print("Naive Bayes Algorithm accuracy of baseline:", (nltk.classify.accuracy(classifier_baseline, testing_set_baseline)))

Naive Bayes Algorithm accuracy of baseline: 0.7101588830043332


In [5]:
training_set_bigrams = featuresets_bigrams[:2077]
testing_set_bigrams = featuresets_bigrams[2077:]
classifier_bigrams = nltk.NaiveBayesClassifier.train(training_set_bigrams)
print("Naive Bayes Algorithm accuracy of bigrams:", (nltk.classify.accuracy(classifier_bigrams, testing_set_bigrams)))

Naive Bayes Algorithm accuracy of bigrams: 0.7580645161290323


In [6]:
training_set_content = featuresets_content[:2077]
testing_set_content = featuresets_content[2077:]
classifier_content = nltk.NaiveBayesClassifier.train(training_set_content)
print("Naive Bayes Algorithm accuracy of content words:", (nltk.classify.accuracy(classifier_content, testing_set_content)))


Naive Bayes Algorithm accuracy of content words: 0.7032980259990371


In [7]:
training_set_bigrams_content = featuresets_bigrams_content[:2077]
testing_set_bigrams_content = featuresets_bigrams_content[2077:]
classifier_bigrams_content = nltk.NaiveBayesClassifier.train(training_set_bigrams_content)
print("Naive Bayes Algorithm accuracy of bigrams (only content words)", (nltk.classify.accuracy(classifier_bigrams_content, testing_set_bigrams_content)))


Naive Bayes Algorithm accuracy of bigrams (only content words) 0.7580645161290323
